In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm

pd.set_option('display.max_columns', 500)

In [2]:
#read train data
train = pd.read_csv('../data/generated/train_eng.csv')
test = pd.read_csv('../data/generated/test_eng.csv')

#extract labels from train
labels = train['label']

#remove labels from train
train.drop(columns=['label'], inplace=True)

print(train.values.shape)
print(labels.values.shape)

(24840, 159)
(24840,)


In [3]:
train.head()

,Cat_DesandSal,Cat_Friday,Cat_Hour_bucket_0,Cat_Hour_bucket_1,Cat_Hour_bucket_2,Cat_Hour_bucket_3,Cat_MailType_multipart/alternative,Cat_MailType_multipart/idm,Cat_MailType_multipart/mixed,Cat_MailType_multipart/related,Cat_MailType_nan,Cat_MailType_text/html,Cat_MailType_text/plain,Cat_Monday,Cat_Saturday,Cat_Sunday,Cat_Thursday,Cat_Trim_1,Cat_Trim_2,Cat_Trim_3,Cat_Trim_4,Cat_Tuesday,Cat_Wednesday,Cat_Year_2012,Cat_Year_2013,Cat_Year_2014,Cat_Year_2015,Cat_Year_2016,Cat_Year_2017,Cat_Year_2018,Cat_Year_2019,Cat_bcced,Cat_ccs,Cat_ccs_2,Cat_designation,Cat_forum_sender,Cat_forum_sender_only,Cat_promo_and_update_sender,Cat_promo_only_sender,Cat_promo_sender,Cat_salutations,Cat_social_sender,Cat_social_sender_only,Cat_update_only_sender,Cat_update_sender,CountByMonth,CountByWeek,Null_sender,SenderTimeZoneDif,chars_in_body,chars_in_body_2,chars_in_subject,chars_in_subject_2,gapByMonth_Day_mean,gapByMonth_Hour_mean,gapByMonth_Minute_mean,gapByMonth_Week_mean,gapByMonth_chars_in_body_mean,gapByMonth_chars_in_subject_mean,gapByMonth_images_mean,gapByMonth_urls_mean,gapByWeek_Day_mean,gapByWeek_Hour_mean,gapByWeek_Minute_mean,gapByWeek_chars_in_body_mean,gapByWeek_chars_in_subject_mean,gapByWeek_images_mean,gapByWeek_urls_mean,gap_Day_mean,gap_Hour_mean,gap_Minute_mean,gap_Month_mean,gap_Week_mean,gap_Year_mean,gap_chars_in_body_mean,gap_chars_in_subject_mean,gap_images_mean,gap_urls_mean,images,images_2,ratio_gapByMonth_Day_std,ratio_gapByMonth_Hour_std,ratio_gapByMonth_Minute_std,ratio_gapByMonth_Week_std,ratio_gapByMonth_chars_in_body_std,ratio_gapByMonth_chars_in_subject_std,ratio_gapByMonth_images_std,ratio_gapByMonth_urls_std,ratio_gapByWeek_Day_std,ratio_gapByWeek_Hour_std,ratio_gapByWeek_Minute_std,ratio_gapByWeek_chars_in_body_std,ratio_gapByWeek_chars_in_subject_std,ratio_gapByWeek_images_std,ratio_gapByWeek_urls_std,ratio_gap_Day_std,ratio_gap_Hour_std,ratio_gap_Minute_std,ratio_gap_Month_std,ratio_gap_Week_std,ratio_gap_Year_std,ratio_gap_chars_in_body_std,ratio_gap_chars_in_subject_std,ratio_gap_images_std,ratio_gap_urls_std,senderByMonth_Day_mean,senderByMonth_Day_std,senderByMonth_Hour_mean,senderByMonth_Hour_std,senderByMonth_Minute_mean,senderByMonth_Minute_std,senderByMonth_Week_mean,senderByMonth_Week_std,senderByMonth_chars_in_body_mean,senderByMonth_chars_in_body_std,senderByMonth_chars_in_subject_mean,senderByMonth_chars_in_subject_std,senderByMonth_images_mean,senderByMonth_images_std,senderByMonth_urls_mean,senderByMonth_urls_std,senderByWeek_Day_mean,senderByWeek_Day_std,senderByWeek_Hour_mean,senderByWeek_Hour_std,senderByWeek_Minute_mean,senderByWeek_Minute_std,senderByWeek_chars_in_body_mean,senderByWeek_chars_in_body_std,senderByWeek_chars_in_subject_mean,senderByWeek_chars_in_subject_std,senderByWeek_images_mean,senderByWeek_images_std,senderByWeek_urls_mean,senderByWeek_urls_std,sender_Day_mean,sender_Day_std,sender_Hour_mean,sender_Hour_std,sender_Minute_mean,sender_Minute_std,sender_Month_mean,sender_Month_std,sender_Week_mean,sender_Week_std,sender_Year_mean,sender_Year_std,sender_chars_in_body_mean,sender_chars_in_body_std,sender_chars_in_subject_mean,sender_chars_in_subject_std,sender_count,sender_depth,sender_images_mean,sender_images_std,sender_urls_mean,sender_urls_std,urls,urls_2
0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,24.0,6.0,0,0.0,4,16,102,10404,12.235294,-3.117647,-3.058824,1.294118,-558.058824,56.205882,-0.088235,-1.058824,3.000000,-0.714286,-7.142857,0.000000,42.142857,0.000000,0.000000,12.286993,-5.447494,-7.942721,1.482697,7.537589,-1.489857,-39307.581742,51.355012,-9.502387,-79.059069,0,0,1.381591,-0.707092,-0.183317,1.020591,-0.171499,2.261844,-0.171499,-0.171499,1.388730,-0.641941,-0.420793,0.000000,1.317516,0.000000,0.000000,1.426592,-0.890395,-0.479093,0.459468,0.536713,-1.618777,-0.949191,1.981859,-0.836731,-0.972081,15.764706,8.855948,3.117647,4.409114,20.058824,16.685943,32.705882,1.268008,562.058824,3.254014e+03,45.794118,24.84

In [4]:
def PowerData(df):
    '''
    add for all none categorical data it's square
    '''
    square_columns = [col for col in df.columns[~df.columns.str.startswith('Cat_')]]
    
    print('Nb columns to square :', len(square_columns))
    
    for col in square_columns:
        df[f'{col}_x2'] = df[col]**2
        df[f'{col}_x3'] = df[col]**3
        df[f'{col}_x0_5'] = df[col]**0.5
        
    return df

In [5]:
def NormalizeData(train, CVorTest, PCA_comp = None, ScaleCat = False):
    '''
    Normalize data using a standard scaler
    train:
        dataframe that will be use to fit and transformed by the scaler and PCA
    CVorTest:
        dataframe that will be transformed the scaler and PCA
    PCA_comp:
        Number of PCA components to keep, if None, PCA not applied
    ScaleCat:
        Scale or not the categorical columns with the standard scaler
    '''
    
    sc = StandardScaler()
    
    if ScaleCat:
        scale_columns = train.columns
    else:
        scale_columns = [col for col in train.columns[~train.columns.str.startswith('Cat_')]]
          
    #perform feature scaling    
    print('Nb columns to scale :', len(scale_columns))
    train.loc[:, scale_columns] = sc.fit_transform(train.loc[:, scale_columns]) 
    CVorTest.loc[:, scale_columns] = sc.transform(CVorTest.loc[:, scale_columns]) 
    
    if PCA_comp is None:
        return train.values, CVorTest.values
    
    pca = PCA(PCA_comp)
    train = pca.fit_transform(train)
    CVorTest = pca.transform(CVorTest)
    
    return train, CVorTest

In [6]:
#train = PowerData(train)
#test = PowerData(test)

print('train shape :', train.shape, 'test shape', test.shape)

train, test = NormalizeData(train, test, None, False)


train shape : (24840, 159) test shape (10647, 159)
Nb columns to scale : 114


In [7]:
display(train)
display(train.shape)

array([[ 0.        ,  0.        ,  1.        , ...,  1.59339039,
        -0.65640229, -0.02656295],
       [ 0.        ,  0.        ,  0.        , ...,  0.1801211 ,
         0.0760027 , -0.01660475],
       [ 1.        ,  0.        ,  1.        , ...,  0.18217618,
         1.17461017,  0.03567575],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.1801211 ,
        -0.22361752, -0.02308581],
       [ 0.        ,  0.        ,  0.        , ...,  1.43698393,
        -0.65640229, -0.02656295],
       [ 0.        ,  0.        ,  0.        , ..., -0.43831867,
        -0.35678207, -0.02489639]])

(24840, 159)

In [8]:
model = LogisticRegressionCV(cv=5, scoring='f1_macro', multi_class='multinomial', max_iter=3000, n_jobs=4)

In [9]:
model.fit(train, labels.values)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=3000, multi_class='multinomial', n_jobs=4,
                     penalty='l2', random_state=None, refit=True,
                     scoring='f1_macro', solver='lbfgs', tol=0.0001, verbose=0)

In [10]:
model.scores_

{0: array([[0.7866859 , 0.86634165, 0.89845224, 0.91271397, 0.91713022,
         0.92020704, 0.9201156 , 0.91996448, 0.9202313 , 0.91931595],
        [0.78530287, 0.86129845, 0.89815787, 0.90876311, 0.9154316 ,
         0.91772506, 0.91888454, 0.91876066, 0.9188089 , 0.9188089 ],
        [0.79713305, 0.86347464, 0.89656846, 0.91247299, 0.91691104,
         0.92070212, 0.92331463, 0.92274827, 0.92344575, 0.92397823],
        [0.79785011, 0.86524338, 0.89925648, 0.91655836, 0.92232461,
         0.92338071, 0.9234952 , 0.9227805 , 0.92245732, 0.92271036],
        [0.79216989, 0.86996964, 0.90499081, 0.91521947, 0.91976128,
         0.92301375, 0.92253572, 0.92175101, 0.92210501, 0.92152213]]),
 1: array([[0.7866859 , 0.86634165, 0.89845224, 0.91271397, 0.91713022,
         0.92020704, 0.9201156 , 0.91996448, 0.9202313 , 0.91931595],
        [0.78530287, 0.86129845, 0.89815787, 0.90876311, 0.9154316 ,
         0.91772506, 0.91888454, 0.91876066, 0.9188089 , 0.9188089 ],
        [0.79713305

In [11]:
model.score(train, labels.values)

/Users/seb/opt/anaconda3/envs/emailClass/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.9272956346598443

In [12]:
preds = model.predict(test)

In [13]:
submission = pd.read_csv('../data/sample_submission.csv')
submission['label'] = preds
submission.head(5)

,ID,label
0,0,2
1,1,3
2,2,3
3,3,0
4,4,0


In [14]:
#build submission file
submission.to_csv('../data/generated/submission_logistic_regression.csv', index=False)

In [15]:
#get logits proba for ensembling later
testpred = model.predict_log_proba(test)

In [16]:
#try to uniformized value to ensemble with other models such as NN
testpred / np.sum(testpred, -1).reshape(-1, 1)

/Users/seb/opt/anaconda3/envs/emailClass/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[1.62703368e-01, 5.65812042e-01, 2.41505143e-04, 2.71243085e-01],
       [3.53036596e-02, 2.06357374e-01, 7.57182248e-01, 1.15671824e-03],
       [1.31291186e-01, 5.74004242e-01, 2.94704505e-01, 6.68689760e-08],
       ...,
       [4.95063258e-02, 2.45327602e-01, 7.03744983e-01, 1.42108917e-03],
       [2.89109996e-02, 2.67085588e-01, 6.97674691e-01, 6.32872136e-03],
       [5.63882699e-07, 4.21709312e-01, 4.55654915e-01, 1.22635209e-01]])